In [2]:
import os

from utils import date_from_file_name
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from lxml import etree # type: ignore <- pylance milně hlásí chybu
from pathlib import Path
import time
import sys

from utils import date_from_file_name

os.chdir('..')

In [3]:
namespaces = {
    'm': 'istp:opendata:schemas:MereniSeznam:v1', 
    'd': 'istp:opendata:schemas:DatovaSada:v1'      
}

xml_file = Path('kod/data/data_z_mericich_pristroju/Data z měřících přístrojů 27-08-2022.xml')
tree = etree.parse(xml_file)
datovy_obsah = tree.find('d:DatovyObsah', namespaces)
mereni_seznam_element = datovy_obsah[0]

OSError: Error reading file 'kod\data\data_z_mericich_pristroju\Data z měřících přístrojů 27-08-2022.xml': failed to load external entity "kod/data/data_z_mericich_pristroju/Data z měřících přístrojů 27-08-2022.xml"

In [42]:
len(mereni_seznam_element)

391

In [33]:
list(tree.getroot().iter())

[<Element {istp:opendata:schemas:DatovaSada:v1}DatovaSada at 0x1ed96720440>,
 <Element {istp:opendata:schemas:DatovaSada:v1}DatovyObsahInfo at 0x1ed9673c3c0>,
 <Element {istp:opendata:schemas:DatovaSada:v1}ID at 0x1ed9673ef40>,
 <Element {istp:opendata:schemas:DatovaSada:v1}CasVytvoreni at 0x1ed9673fa40>,
 <Element {istp:opendata:schemas:DatovaSada:v1}CasovePokrytiZacatek at 0x1ed9673c840>,
 <Element {istp:opendata:schemas:DatovaSada:v1}CasovePokrytiKonec at 0x1ed9673d300>,
 <Element {istp:opendata:schemas:DatovaSada:v1}DatovyObsah at 0x1ed9673ee40>,
 <Element {istp:opendata:schemas:MereniSeznam:v1}MereniSeznam at 0x1ed9673d140>,
 <Element {istp:opendata:schemas:MereniSeznam:v1}Mereni at 0x1ed9673ecc0>,
 <Element {istp:opendata:schemas:MereniSeznam:v1}CisloProtokolu at 0x1ed9673d040>,
 <Element {istp:opendata:schemas:MereniSeznam:v1}DatumProhlidky at 0x1ed9673f940>,
 <Element {istp:opendata:schemas:MereniSeznam:v1}Stanice at 0x1ed9673d540>,
 <Element {istp:opendata:schemas:MereniSeznam

In [4]:
os.listdir()

['.git',
 '.gitignore',
 '.venv',
 'kod',
 'LICENSE',
 'poznamky',
 'prezentace',
 'sablona',
 'spustit_powershell.lnk',
 'zdroje']

In [ ]:
def get_date_state(date, DOWNLOAD_DIR):
    downloaded = {}


In [6]:
DOWNLOAD_DIR = Path('kod/data/datove_sady_stk')
list(DOWNLOAD_DIR.glob('*31-*'))


[WindowsPath('kod/data/datove_sady_stk/Prohlídky vozidel STK a SME za 31-01-2019.xml'),
 WindowsPath('kod/data/datove_sady_stk/Prohlídky vozidel STK a SME za 31-01-2020.xml'),
 WindowsPath('kod/data/datove_sady_stk/Prohlídky vozidel STK a SME za 31-01-2021.xml'),
 WindowsPath('kod/data/datove_sady_stk/Prohlídky vozidel STK a SME za 31-01-2022.xml'),
 WindowsPath('kod/data/datove_sady_stk/Prohlídky vozidel STK a SME za 31-01-2023.xml'),
 WindowsPath('kod/data/datove_sady_stk/Prohlídky vozidel STK a SME za 31-01-2024.xml'),
 WindowsPath('kod/data/datove_sady_stk/Prohlídky vozidel STK a SME za 31-03-2019.xml'),
 WindowsPath('kod/data/datove_sady_stk/Prohlídky vozidel STK a SME za 31-03-2020.xml'),
 WindowsPath('kod/data/datove_sady_stk/Prohlídky vozidel STK a SME za 31-03-2021.xml'),
 WindowsPath('kod/data/datove_sady_stk/Prohlídky vozidel STK a SME za 31-03-2022.xml'),
 WindowsPath('kod/data/datove_sady_stk/Prohlídky vozidel STK a SME za 31-03-2023.xml'),
 WindowsPath('kod/data/datove_sa